In [1]:
# Importing the libraries

import pandas as pd
import numpy as np
import warnings
import sys
sys.path.append("..")
warnings.filterwarnings('ignore')
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

In [2]:
# Importing some scripts

from scripts.data_vizualization import Data_Viz 
from scripts.data_cleaning import DataCleaner
from scripts.data_transformation import DataTransformer

DC = DataCleaner()
DV = Data_Viz()
DT = DataTransformer()

In [3]:
# Importing the train data and check for outliers

train = pd.read_csv('../data/train_data_clean.csv',index_col='Date')
DV.summ_columns(train)

,variables,missing_count,missing_percent_(%),data_type,unique_values
0,Store,0,0.0,int64,1115
1,DayOfWeek,0,0.0,int64,7
2,Sales,0,0.0,float64,13194
3,Customers,0,0.0,float64,1423
4,Open,0,0.0,int64,2
5,Promo,0,0.0,int64,2
6,StateHoliday,0,0.0,object,4
7,SchoolHoliday,0,0.0,int64,2
8,StoreType,0,0.0,object,4
9,Assortment,0,0.0,object,3


In [19]:
# Creating a new column

train.loc[train['DayOfMonth'] >= 20,'DayStatus'] = 'End of Month' 
train.loc[(train['DayOfMonth'] < 20) & (train['DayOfMonth'] > 10),'DayStatus'] = 'Mid of Month' 
train.loc[train['DayOfMonth'] < 10,'DayStatus'] = 'Begining of Month' 

In [22]:
# Finding out if the column worked as planned

train['DayStatus'].value_counts()

End of Month         380995
Begining of Month    334849
Mid of Month         301365
Name: DayStatus, dtype: int64

In [25]:
# Lets see the catagorical and numerical columns and list out all the numerical columns

categorical_col, numerical_col = DT.sep_cat_num(train)
numerical_col.head()

,Store,DayOfWeek,Sales,Customers,Open,Promo,SchoolHoliday,CompetitionDistance,Promo2,Year,Month,DayOfMonth,Weekday
Date,,,,,,,,,,,,,
2015-07-31,1,5,5263.0,555.0,1,1,1,1270.0,0,2015,7,31,1
2015-07-31,2,5,6064.0,625.0,1,1,1,570.0,1,2015,7,31,1
2015-07-31,3,5,8314.0,821.0,1,1,1,14130.0,1,2015,7,31,1
2015-07-31,4,5,13995.0,596.0,1,1,1,620.0,0,2015,7,31,1
2015-07-31,5,5,4822.0,559.0,1,1,1,29910.0,0,2015,7,31,1


In [26]:
# Lets see the catagorical columns

categorical_col.head()

,StateHoliday,StoreType,Assortment,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2SinceWeek,Promo2SinceYear,PromoInterval,DayStatus
Date,,,,,,,,,
2015-07-31,No Holiday,c,a,9.0,2008.0,Not Available,Not Available,Not Available,End of Month
2015-07-31,No Holiday,a,a,11.0,2007.0,13.0,2010.0,"Jan,Apr,Jul,Oct",End of Month
2015-07-31,No Holiday,a,a,12.0,2006.0,14.0,2011.0,"Jan,Apr,Jul,Oct",End of Month
2015-07-31,No Holiday,c,c,9.0,2009.0,Not Available,Not Available,Not Available,End of Month
2015-07-31,No Holiday,a,a,4.0,2015.0,Not Available,Not Available,Not Available,End of Month
